## Replacer

### This tutorial shows how to replace any text in a whole analysis. 
Be careful, only use real key words to replace, otherwise you might replace also words or word parts you did not mean to.



### First connect to the analysis and indicate what should be replaced
It is recommended to use the ID of the respective analysis.

In [36]:
import json
from pycelonis import get_celonis

c = get_celonis("URL to the team in which you are working.", "Specify a valid API key for the cloud team.")
analysis = c.analyses.find("Name or ID of the analysis.")

# enter as many replacements as you want and separate them with a comma
replacements ={
    'old_word_1' : 'new_word_1',
    'old_word_2' : 'new_word_2'
}

2020-02-17 08:20:12 - pycelonis: Login successful! Hello s.matthaei@celonis.com


### Get the current version of the published analysis, the drafted one from the edit mode and the saved formulas

In [40]:
num_replacements = 0
doc_p = analysis.published.data
doc_d = analysis.draft.data
kpis = analysis.saved_formulas

### Replace in the formulas

In [38]:
for kpi in kpis:
    for key, val in replacements.items():
        num_replacements = num_replacements + kpi.data["name"].count(key)
        num_replacements = num_replacements + kpi.data["template"].count(key)
        num_replacements = num_replacements + kpi.data["description"].count(key)
        
        name = kpi.data["name"].replace(key, val)
        template = kpi.data["template"].replace(key, val)
        description = kpi.data["description"].replace(key, val)
        parameters = kpi.data["parameters"]
        
        kpi.delete()
        analysis.create_saved_formula(name=name, description=description, template=template, parameters=parameters)       
        
print('Replacements in the formulas: ' + str(num_replacements))

Replacements in the formulas: 78


### Replace in the published and drafted analysis

In [39]:
json_doc_dump_p = json.dumps(doc_p, ensure_ascii=False)
json_doc_dump_p = json_doc_dump_p.replace("â¬", "€")

json_doc_dump_d = json.dumps(doc_d, ensure_ascii=False)
json_doc_dump_d = json_doc_dump_d.replace("â¬", "€")

for key, val in replacements.items():
    num_replacements += json_doc_dump_p.count(key)
    json_doc_dump_p = json_doc_dump_p.replace(key, val)
    json_doc_dump_d = json_doc_dump_d.replace(key, val)
    
json_doc_dump_p = json_doc_dump_p.replace("â¬", "€")
json_doc_dump_d = json_doc_dump_d.replace("â¬", "€")

doc_p = json.loads(json_doc_dump_p)
doc_d = json.loads(json_doc_dump_d)

analysis.draft.data = doc_d
analysis.published.data = doc_p

print('Overall replacements: ' + str(num_replacements))

Overall replacements: 143
